## A simple notebook showing overfitting for higher model complexity.

A polinomial fit is evidently not a good model, but it is used because of its mathematical simplicity

(Bishop uses a similar approach to show bias-variance trade-off and/or overfitting)

Prepared for a demo class in a peruvian university

In [78]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from ipywidgets import interact,IntSlider,Layout,VBox,Checkbox
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

MAX_DEGREE = 6
NOISE_LEVEL = 0.4
N_POINTS = int(2*MAX_DEGREE)
TEST_SIZE = 0.4

SEED = 0
np.random.seed(SEED)



def plot_polynomial_fit(degree,test_visible):
    # reproducibility:
    np.random.seed(SEED)

    #domain:
    min_x,max_x = -1.5,1.5
    x = np.linspace(min_x, max_x, N_POINTS)

    # real function:
    def fun(x):
        return  0.6*(x+0.1)**3 + np.random.normal(0, NOISE_LEVEL, x.shape)
    
    y = fun(x)
    # train test splitting and output evaluation for synthetic data:
    # to avoid extrapolation, make sure first and last points are in train:
    # this is just to focus around the point intended in the class:
    indices = range(N_POINTS)
    
    # Split using indices
    train_idx, test_idx = train_test_split(indices, test_size=TEST_SIZE, random_state=SEED)
    #make sure that first and last points are in the training set so that we are not extrapolating:
    train_idx += [0,N_POINTS-1]
    train_idx = list(set(train_idx))
    # if either first or last index has been moved to train set, make sure they 
    # are not in the test set then:
    test_idx = [v for v in test_idx if v not in train_idx]
    # Use the indices to get the train and test sets
    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    degree_range = (1, MAX_DEGREE, 1) #start,end and step

    # do a pass just to get min and max for y axis to keep points constants in the canvas later:
    # kept separate to improve readabilty of program:
    min_y = min(min(y_train),min(y_test))
    max_y = max(max(y_train),max(y_test))
    for d in range(degree_range[0],degree_range[1]+1,degree_range[2]):
        # Fit polynomial
        coeffs,residuals, _, _, _  = np.polyfit(x_train, y_train, d,full=True,rcond=np.finfo(float).eps/10)
        poly_func= np.poly1d(coeffs)
        x1 = np.linspace(min_x, max_x, 5*N_POINTS)
        y1 = poly_func(x1)
        min_y,max_y = min(min_y,min(y1)),max(max_y,max(y1))
  
    # fits mapping:
    fits_dict = {}
    train_MSE_dict = {}
    test_MSE_dict = {}
    colors = []
    # build fits:
    

    # Create plot
    # Create a figure with 1 row and 2 columns
    fig = plt.figure(figsize=(16, 6))
    plt.suptitle('UP demo class - 12-Feb-25 - © Paul Arriz Tisoc', fontsize=16, fontfamily="serif", fontweight="bold")

    # Define grid spec: first subplot will take more space (e.g. 2/3 of the width)
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # The first subplot is twice as wide as the second

    # Create subplots with the custom grid spec
    axs = [plt.subplot(g) for g in gs]

    # iteract of ipywidgets includes last element but range doesnt
    # which casues with range below to be a little ugly:
    for deg in range(degree_range[0],degree_range[1]+1,degree_range[2]):   

        # Fit polynomial
        coeffs,residuals, _, _, _  = np.polyfit(x_train, y_train, deg,full=True,rcond=np.finfo(float).eps/10)
        
        poly_func= np.poly1d(coeffs)
        # append new fit:
        fits_dict[deg] = poly_func
        # per numpy: residuals – sum of squared residuals of the least squares fit
        # residuals provided before as sum of squared values:
        train_MSE_dict[deg] = np.sqrt(residuals)/len(x_train)

        #the outputs calculated using the "trained" model
        y_pred_train = poly_func(x_train)
        y_pred_test = poly_func(x_test)
        # the error between the real values for x_test, time N to make it sum
        #as in redisuals
        test_MSE_dict[deg] = mean_squared_error(y_true=y_test,y_pred=y_pred_test)
        
        if deg == degree:
            colors.append('red')
            #plot the given left side chart
            #axs[0].scatter(x_train, y_train, label='Noisy Data')
            axs[0].plot(x_train, y_train, 'x', color='magenta', markersize = 16, label='Train - Real Data')
            axs[0].plot(x_train, y_pred_train, 'o', color='magenta', label='Train - Predicted')
            if test_visible:
                axs[0].plot(x_test, y_test, 'x', color='lightgreen',markersize = 16, label='Test  - Real Data')
                axs[0].plot(x_test, y_pred_test, 'o', color='green', label='Test  - Predicted')
            # plot the actual model including point inbeweeen:
            x1 = np.linspace(min_x, max_x, 5*N_POINTS)
            y1 = poly_func(x1)
            axs[0].plot(x1, y1,'k--', label=f'Model fit (grado {deg})')

            axs[0].set_title(f' A fit of some data points with some simplistic model')
            axs[0].set_xlabel('x : The data we have as predictor(s) - the input to the model')
            axs[0].set_ylabel('y : The Value we attempt to predict - the output of the model')


        else:
            colors.append('black')
    # there are better ways of doing this, but this is simpler to explain:
    # still part of the curve might be about of the boundaries

    axs[0].set_ylim(1.1*min_y,1.1*max_y)

    #plot the accuracies
    axs[1].scatter(train_MSE_dict.keys(), train_MSE_dict.values(), color = colors, label='Train data - error')
    axs[1].plot(train_MSE_dict.keys(), train_MSE_dict.values(),"k--")
    if test_visible:
        axs[1].scatter(test_MSE_dict.keys(), test_MSE_dict.values(), color = [c if c == 'red' else 'green' for c in colors], label='Test data - error')
        axs[1].plot(test_MSE_dict.keys(), test_MSE_dict.values(),"g--")
    axs[1].axvline(x=degree, color='red', linestyle='--', label='current complexity (grade)')
    axs[1].set_title(f' Error of the model for different complexities')
    axs[1].set_xlabel('<-- lower -- Model Complexity  -- higher-->')
    axs[1].set_ylabel('Mean Squared Error')
    axs[1].set_yscale("log")
    axs[1].legend()

    fig.tight_layout()
    axs[0].legend(loc='upper left',)

    #accuracy_score()

    #fig.show()

# Create interaction
common_layout = Layout(width='600px', margin='0 0 0 0')
interact(plot_polynomial_fit, 
         degree=IntSlider(description='Model Complexity (degree)', 
                          min=1, 
                          max=MAX_DEGREE, 
                          step=1, 
                          value=5, 
                          layout=common_layout,  # Set the overall width
                          style={'description_width': '200px'},  # Increase the description width
                          height='60px'),
         test_visible=Checkbox(
            description='Show test data',
            value=False,
            layout=common_layout,  # Set the overall width
        ))

interactive(children=(IntSlider(value=5, description='Model Complexity (degree)', layout=Layout(margin='0 0 0 …

<function __main__.plot_polynomial_fit(degree, test_visible)>

In [ ]:
#from sklearn.datasets import fetch_california_housing
#housing = fetch_california_housing()

In [ ]:
#X_df = pd.DataFrame(data = housing['data'],columns= housing['feature_names'])
#y = pd.Series(data = housing['target'], name = housing['target_names'][0])

In [ ]:
#housing['feature_names']

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

['MedHouseVal']